pip install xgboost


In [89]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn import linear_model
from sklearn.metrics import recall_score, accuracy_score, f1_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from keras.layers import Input,Dense
import streamlit as st
import joblib
import keras
from keras import layers

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_csv('fraudTrain.csv')

In [ ]:
df_fraud = df[df['is_fraud']==1]

In [ ]:
df_fraud.shape

In [ ]:
df_fraud.info()

In [ ]:
fig = px.histogram(data_frame = df_fraud,
    x = 'amt',
    title = 'Distribution of Fraud vs Amount'
)
fig.show()

In [ ]:
gender_fraud = px.box(data_frame=df_fraud,
                      x='gender',
                      y='amt',
                      title ='Gender vs Amount')
gender_fraud.show()

In [ ]:
gender_fraud = px.box(data_frame=df_fraud,
                      x='state',
                      y='amt',
                      title ='State vs Amount')
gender_fraud.show()

In [ ]:
df_fraud.state.unique()

In [ ]:
df_fraud.loc[:,['amt','state']].groupby(['state']).count().reset_index().sort_values(by='amt', ascending=False)

In [ ]:
def fraud_state(state):
    if state in ['NY','TX','PA','CA','OH']:
        return 1
    else:
        return 0

In [ ]:
top_five_fraud_states = df.state.apply(fraud_state)

In [ ]:
df['top_five_fraud_states']= top_five_fraud_states

In [ ]:
df.info()

In [ ]:
df.category.unique()

In [ ]:
category_amt = px.box(data_frame=df_fraud,
                      x='category',
                      y='amt',
                      title='Category Amt')
category_amt.show()

In [ ]:
df = pd.get_dummies(columns=['category'], data=df, dtype=float)

In [ ]:
df = pd.get_dummies(columns=['gender'], data=df, dtype=float)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#merch = px.scatter(data_frame=df,x='merch_lat',y='merch_long', color='is_fraud', title='Merch Lat vs Long')
#merch.show()

In [ ]:
#fig = px.choropleth(data_frame=df_fraud, color='amt', locationmode='USA-states',
                           # locations='state',
                            #scope='usa',
                           #range_color=(0, 12),
                          #)
#fig.show()

In [ ]:
X = df.loc[:,['amt', 'top_five_fraud_states', 'city_pop', 'gender_M', 'merch_lat', 'merch_long', 
             'category_entertainment', 'category_food_dining', 'category_gas_transport','category_grocery_net',
             'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net',
              'category_misc_pos', 'category_personal_care', 'category_shopping_net', 'category_shopping_pos', 'category_travel',]]

In [ ]:
y = df.loc[:,['is_fraud']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.33, random_state=42, stratify=y)
#stratify when we have class imbalance.

In [ ]:
scale = StandardScaler()

In [ ]:
X_train=scale.fit_transform(X_train)

In [ ]:
joblib.dump(scale,'scaler.joblib')

In [ ]:
X_test = scale.transform(X_test)

In [ ]:
lr = linear_model.LogisticRegression(C=0.01, class_weight = 'balanced')

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
r_train_score = recall_score(y_train, lr.predict(X_train))
r_test_score = recall_score(y_test, lr.predict(X_test))
print(r_train_score, r_test_score)

In [ ]:
a_train_score = accuracy_score(y_train, lr.predict(X_train))
a_test_score = accuracy_score(y_test, lr.predict(X_test))
print(a_train_score, a_test_score)

In [ ]:
p_train_score = precision_score(y_train, lr.predict(X_train))
p_test_score = precision_score(y_test, lr.predict(X_test))
print(p_train_score, p_test_score)

In [ ]:
ConfusionMatrixDisplay.from_estimator(lr, X_test, y_test)

In [ ]:
random_forest = RandomForestClassifier(max_depth=5, random_state=0, class_weight='balanced', n_estimators=200)
random_forest.fit(X_train,y_train)

In [ ]:
RF_r_train_score = recall_score(y_train, random_forest.predict(X_train))
RF_r_test_score = recall_score(y_test, random_forest.predict(X_test))
print(RF_r_train_score, RF_r_test_score)

In [ ]:
RF_p_train_score = precision_score(y_train, random_forest.predict(X_train))
RF_p_test_score = precision_score(y_test, random_forest.predict(X_test))
print(RF_p_train_score, RF_p_test_score)

In [ ]:
bst = XGBClassifier(n_estimators=25, max_depth=5, learning_rate=1, objective='binary:logistic', scale_pos_weight=2)
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)

In [ ]:
G_p_train_score = precision_score(y_train, bst.predict(X_train))
G_p_test_score = precision_score(y_test, bst.predict(X_test))
print(G_p_train_score, G_p_test_score)

In [ ]:
G_r_train_score = recall_score(y_train, bst.predict(X_train))
G_r_test_score = recall_score(y_test, bst.predict(X_test))
print(G_r_train_score, G_r_test_score)

In [ ]:
f1_r_train_score = f1_score(y_train, bst.predict(X_train))
f1_r_test_score = f1_score(y_test, bst.predict(X_test))
print(f1_r_train_score, f1_r_test_score)

In [ ]:
bst.save_model('model_file_name.json')

In [ ]:
#Streamlit

In [ ]:
@st.cache
def load_data():
    data = df
    numeric_data = data.select_dtypes(['float','int'])
    numeric_cols = numeric_data.columns
    return data, numeric_cols

In [ ]:
#data, numeric_cols = load_data()

In [ ]:
st.title('Fraud Detection')

In [ ]:
st.write(data)

In [ ]:
#Deep Learning

In [ ]:
encoding_dim = 5

In [ ]:
encoded = layers.Dense(encoding_dim, activation='relu')(keras.Input(X))

In [ ]:
decoded = layers.Dense(784, activation='sigmoid')(encoded)

In [ ]:
autoencoder = keras.Model(df, decoded)

In [ ]:
encoder = keras.Model(df, encoded)

In [ ]:
# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
x_train = keras.Input(X_train.astype('float32') / 255)
x_test = keras.Input(X_test.astype('float32') / 255)
x_train = keras.Input(X_train.reshape((len(X_train), np.prod(X_train.shape[1:]))))
x_test = kera.Input(X_test.reshape((len(X_test), np.prod(X_test.shape[1:]))))
print(X_train.shape)
print(X_test.shape)

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))